### This script transforms the measured and synthetic data into the formar required to perform the PV calculations

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
from sklearn.metrics import mean_squared_error
import pickle
from datetime import timedelta as timedelta
from dateutil.relativedelta import relativedelta
# import datetime as dt

from support import *
from features.feature_ts import genX
from experiment.algorithms.cluster_prep import *
from Gauss_fit_functions import extractFIT, extractToPs , gauss, straight_line

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\os.py
c:\Users\Jason\thesis_project


# Load the measured data

In [2]:
X = genX([1994,2014], drop_0 = True)

dropping all zero rows


## Load IDs of profiles with the necesary amount of High and Low Season data

In [3]:
profileIDs = pd.read_pickle("ProfileIDs.pkl")

In [4]:
# # Collect only winter weekday profiles from dataset

# X.reset_index(inplace = True)

# df = X.copy()

# # Extract Season
# df['month'] = df.date.dt.month
# df['season'] = df['month'].apply(lambda x: 'winter' if x in [6, 7, 8] else 'summer') 
# df_summer = df[df['season'] == 'summer'] # Create dataframe with all the winter months, excluding weekends


# # Extract Weekdays
# weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
# df_summer['day_names'] = df_summer.date.dt.day_name()
# df_summer['daytype'] = df_summer.day_names.where(~df_summer.day_names.isin(weekdays), 'weekday')
# df_summer.drop(['day_names'], axis = 1, inplace = True)
# df_summer_weekdays  = df_summer[df_summer['daytype'] == 'weekday'] # Create dataframe with only weekdays
# df_summer_weekdays.drop(['month', 'season','daytype'], axis = 1, inplace = True)
# df_summer_weekdays.set_index(['ProfileID','date'], inplace=True)

# mean = df_summer_weekdays.groupby(["ProfileID"]).mean()

# # Test 1: Energy between 9-5 
# i = np.arange(9,17)
# sum_df = pd.DataFrame(index = mean.index, columns = ["9_to_5", 'total'])
# sum_df = sum_df.fillna(0)
# for x in i:
#     temp = mean.iloc[:,x]
#     sum_df['9_to_5'] = sum_df['9_to_5'] + temp

# sum_df['total'] =  mean.sum(axis = 1)

# sum_df["percentage"] = (sum_df["9_to_5"]/sum_df["total"]) * 100

# def check_9_to_5(df):
#     if df > 50:
#         return 'B'
#     else: 
#         return 'R'

# sum_df['Labels'] = sum_df['percentage'].apply(check_9_to_5)

# # 173 households labeled as Businesses for the 9-to-5 check
# sum_df[sum_df['percentage'] > 50]

# # Tests 2: ToP
# top_df = pd.DataFrame()

# top_df['Time'] = mean.idxmax(axis=1)

# def check_top(df):
#     if int(df) > 9 and int(df) < 17:
#         return 'B'
#     else:
#         return 'R'  

# top_df['Labels'] = top_df['Time'].apply(check_top)

# new_df = pd.DataFrame()

# new_df['ToP'], new_df['Sum'] = top_df['Labels'], sum_df['Labels']

# data_classes = ['R','B']
# d = dict(zip(data_classes, range(0,2)))

# new_df['ToP'],new_df['Sum'] = new_df['ToP'].map(d, na_action='ignore'),new_df['Sum'].map(d, na_action='ignore')

# new_df['Total'] = new_df.sum(axis = 1)

# residential_ids = new_df[new_df['Total'] == 0].index.values # Contains the profile ids that have typical residential behaviour

<ipython-input-4-64b8476c9274>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-64b8476c9274>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [414]:
# df_summer_weekdays.reset_index(inplace = True)

# # Remove all households with less than three winter months
# winter_months_ids = []

# for i in df_summer_weekdays['ProfileID'].unique():
#     if len(df_summer_weekdays[df_summer_weekdays['ProfileID'] == i]) >= 30:

#         winter_months_ids.append(i) # Contains the profile Ids that have a complete winter months data

# summer_months_ids = np.asarray(winter_months_ids)

In [415]:
# profileIDs = np.intersect1d(summer_months_ids, residential_ids)

# cleaned_df = df_summer_weekdays[df_summer_weekdays['ProfileID'].isin(profileIDs)]

# # # Load the previously identified profiles
# # id_identified_profiles = pd.read_pickle('sorted_households_winter_weekdays.pkl') # Read pickle file
# # res = []
# # for key in id_identified_profiles.keys():
# #     res.append(id_identified_profiles[key].tolist())

# # flat_list = list()
# # for sub_list in res:
# #     flat_list += sub_list

# # household_ids = flat_list

# # temp_df = cleaned_df.copy()
# # # temp_df.drop(['date'], axis = 1, inplace = True)
# # temp_df = temp_df.loc[temp_df.ProfileID.isin(household_ids),:]

# # cleaned_df = temp_df

# high_season_profileIDs = pd.read_pickle('ProfileIDs.pkl') # Read pickle file

# temp_ids = cleaned_df['ProfileID'].unique()

# profileIDs = np.intersect1d(high_season_profileIDs, temp_ids)

## Check that every household has one year of data
### if not take the first week of the months data as being representative of the months weekly usage and suplement the measured data

In [8]:
X.reset_index(inplace=True)

In [9]:
measured_DF = X.loc[X.ProfileID.isin(profileIDs),:]

### Create function that replaces missing dates

#### Attempt 1: Broken

In [419]:

# transformed_df = pd.DataFrame()

# for id in profileIDs:
#     data_new =  measured_DF[measured_DF['ProfileID'] == id].copy()

#     # Determine a households range of dates available
#     start_date = data_new['date'].iloc[0]
#     end_date = data_new['date'].iloc[-1]

#     date_range = pd.date_range(start = start_date, end = end_date)

#     # Get list of all the missing dates
#     missing_dates = date_range.difference(data_new['date'])

#     data_new.set_index(['date'], inplace = True)

#     for missing_date in missing_dates:
#         # Check if there is data measured one week prior
#         try:
#             # Get the date one week prior to missing date
#             previous_weeks_date = missing_date - timedelta(weeks=1)
#             data = data_new.loc[previous_weeks_date].tolist()
#             # print('Previous Weeks Date')
#         except KeyError:
#             following_weeks_date = missing_date + timedelta(weeks=1)
#             data = data_new.loc[following_weeks_date].tolist()
        

#         data_new.loc[missing_date] = data # Append list at the bottom
#         data_new = data_new.sort_index()#.reset_index(drop = False)
#         temp_df = data_new

#     transformed_df.append(temp_df)

In [231]:
# Check whether the day is high or low season
    # Check whether value is weekday or weekend
    # If date is weekday:
        # Try to get value from first of month same weekday
        # Else: Try to get any weekday value from month
        # Else: Get weekday value from a month within the same season
    # If date is weekend:
        # Repeat similiar process shown above
# df = data_new.copy()

# df.reset_index(inplace = True)

# df['month'] = df.date.dt.month
# df['season'] = df['month'].apply(lambda x: 'high_season' if x in [6, 7, 8] else 'low_season') 

# # Extract Weekdays
# weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
# weekend = ['Saturday','Sunday']
# df['day_names'] = df.date.dt.day_name()
# # df['day_names'] = df.day_names.where(~df.day_names.isin(weekdays), 'WeekDay')
# # df['day_names'] = df.day_names.where(~df.day_names.isin(weekend), 'WeekEnd') 


In [286]:
# missing_day = missing_date.day_name()
# missing_month = missing_date.month
# data = df[(df['month'] == missing_month) & (df['day_names'] == missing_day)]
# # df = df.set_index(['date'], inplace = True)
# df.loc[missing_date] = data.values[0] # Append list at the bottom
# df = df.sort_index()#.reset_index(drop = False)
# temp_df = df

## Function that replace missing data in measured data

In [11]:
temporary_df = pd.DataFrame(columns = measured_DF.columns)##measured_DF.set_index(['date'])
temporary_df.set_index(['date'], inplace = True)

High_season = [6,7,8]

for id in tqdm(profileIDs):
    data_new =  measured_DF[measured_DF['ProfileID'] == id].copy()

    data_new['month'] = data_new.date.dt.month
    data_new['season'] = data_new['month'].apply(lambda x: 'high_season' if x in [6, 7, 8] else 'low_season') 
    data_new['day_names'] = data_new.date.dt.day_name()

    # Extract Weekdays
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
    weekends = ['Sunday', 'Saturday']
    data_new['day_names'] = data_new.date.dt.day_name()
    data_new['daytype'] = data_new.day_names.apply(lambda x: 'Weekday' if x in weekdays else 'Weekend') 


    # Determine a households range of dates available
    start_date = data_new['date'].iloc[0]
    end_date = data_new['date'].iloc[-1]

    one_year_later = start_date + relativedelta(years = 1)

    if one_year_later < end_date:
        date_range = pd.date_range(start = start_date, end = end_date)
    else:
        date_range = pd.date_range(start = start_date, end = one_year_later)

    # Get list of all the missing dates
    missing_dates = date_range.difference(data_new['date'])

    data_new.set_index(['date'], inplace = True)

    for missing_date in missing_dates:
        missing_day = missing_date.day_name()
        missing_month = missing_date.month
        # Check if there is data measured one week prior

        # Get the date one week prior to missing date
        data = data_new[(data_new['month'] == missing_month) & (data_new['day_names'] == missing_day)]
        if len(data) != 0:
            data_new.loc[missing_date] = data.sample(n = 1).values[0] # Append list at the bottom
            data_new = data_new.sort_index()#.reset_index(drop = False)
            temp_df = data_new
        else:
            x = missing_date.month
            if x in High_season:
                data = data_new[(data_new['season'] == 'high_season') & (data_new['day_names'] == missing_day)]
                if len(data) != 0:
                    data_new.loc[missing_date] = data.sample(n = 1).values[0] # Append list at the bottom
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
                elif missing_day in weekdays:
                    data = data_new[(data_new['season'] == 'high_season') & (data_new['daytype'] == 'Weekday')]
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
                elif missing_day in weekends:
                    data = data_new[(data_new['season'] == 'high_season') & (data_new['daytype'] == 'Weekend')]
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
            else:
                data = data_new[(data_new['season'] == 'low_season') & (data_new['day_names'] == missing_day)]
                if len(data) != 0:
                    data_new.loc[missing_date] = data.sample(n = 1).values[0] # Append list at the bottom
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
                elif missing_day in weekdays:
                    data = data_new[(data_new['season'] == 'low_season') & (data_new['daytype'] == 'Weekday')]
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new
                elif missing_day in weekends:
                    data = data_new[(data_new['season'] == 'low_season') & (data_new['daytype'] == 'Weekend')]
                    data_new = data_new.sort_index()#.reset_index(drop = False)
                    temp_df = data_new

        # print('Previous Weeks Date')

    # temp_df.drop(['month', 'season', 'day_names'], axis = 1, inplace = True)
    temporary_df = temporary_df.append(temp_df)
    

temporary_df.to_csv("Measured_Consumption_Missing_Days_filled.csv")



<ipython-input-11-8342425998b7>:6: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/9993 [00:00<?, ?it/s]

In [13]:
measured_profiles = temporary_df.drop(['month','season','day_names','daytype'], axis =1)

In [14]:
measured_profiles

,ProfileID,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
date,,,,,,,,,,,,,,,,,,,,,
1994-06-18,1,3.691667,0.800000,0.508333,2.183333,0.741667,0.658333,1.991667,0.700000,2.383333,...,1.450000,1.133333,1.666667,3.916667,9.241667,3.275000,6.325000,2.816667,0.925000,2.025000
1994-06-19,1,0.808333,0.500000,2.075000,0.691667,0.325000,2.075000,0.691667,0.316667,3.075000,...,2.125000,2.616667,2.358333,1.141667,7.283333,5.391667,6.441667,9.716667,7.500000,0.841667
1994-06-20,1,1.058333,1.625000,0.341667,2.100000,1.225000,1.058333,4.808333,4.091667,2.083333,...,1.433333,2.641667,2.116667,5.541667,5.275000,6.533333,4.708333,10.433333,9.508333,4.408333
1994-06-21,1,0.425000,2.350000,0.550000,0.291667,2.100000,0.591667,4.066667,2.791667,0.816667,...,3.150000,1.825000,4.016667,1.833333,3.466667,8.875000,10.850000,12.908333,9.241667,2.625000
1994-06-22,1,0.741667,2.325000,0.583333,1.191667,1.225000,0.575000,4.066667,3.350000,0.866667,...,1.500000,0.933333,2.791667,6.291667,6.491667,4.458333,13.125000,10.750000,6.200000,1.350000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-05-12,12029066,14.983334,14.400000,15.083333,12.383333,15.383333,22.400000,30.083333,24.950000,29.983333,...,32.566667,39.366666,40.383333,52.150000,47.833333,39.283333,29.366667,25.083333,17.850000,16.933333
2015-05-13,12029066,15.050000,15.516667,15.066667,16.133333,19.483333,23.883334,22.850000,29.083333,27.833333,...,29.283333,30.883333,29.700000,36.516667,43.583333,38.166667,34.300000,25.200000,21.500000,18.833333
2015-05-14,12029066,18.016666,16.416667,16.283333,15.833333,18.466667,22.850000,24.033333,24.616667,23.333333,...,31.066667,32.600000,35.733333,39.533333,45.883333,37.766666,31.283334,23.750000,18.016667,16.583333


# Check that every user has one year of data

In [405]:
data_new =  measured_DF[measured_DF['ProfileID'] == id]

data_new['month'] = data_new.date.dt.month
data_new['season'] = data_new['month'].apply(lambda x: 'high_season' if x in [6, 7, 8] else 'low_season') 
data_new['day_names'] = data_new.date.dt.day_name()

# Extract Weekdays
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']
weekends = ['Sunday', 'Saturday']
data_new['day_names'] = data_new.date.dt.day_name()
data_new['daytype'] = data_new.day_names.apply(lambda x: 'Weekday' if x in weekdays else 'Weekend') 

<ipython-input-405-c1e1be1eb722>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-405-c1e1be1eb722>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-405-c1e1be1eb722>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-405-c1e

In [410]:
missing_day in weekends

True

In [398]:
data = data_new[(data_new['season'] == 'low_season') & (data_new['day_names'] == missing_day)]

In [404]:
# (data_new['season'] == 'low_season')
data_new['day_names'] == missing_day

4

In [380]:
temporary_df[temporary_df['ProfileID'] == 3]

,ProfileID,0,1,2,3,4,5,6,7,8,...,17,18,19,20,21,22,23,month,season,day_names
date,,,,,,,,,,,,,,,,,,,,,
1997-12-15,3,4.396250,0.673833,0.801250,0.808667,0.813250,0.802917,0.811750,1.048583,3.035583,...,0.733833,0.761167,6.857833,8.509500,3.316750,6.862917,0.888250,12.0,low_season,Monday
1997-12-16,3,0.668250,0.740333,0.741583,0.799250,0.667583,0.800750,1.310417,5.723500,11.440583,...,13.833750,7.974583,6.810083,0.838167,0.751833,0.749083,0.750667,12.0,low_season,Tuesday
1997-12-17,3,0.812167,0.751750,1.382000,6.623917,7.606583,9.273083,0.680833,0.818750,0.757833,...,2.800333,2.352500,0.992833,0.813667,0.747500,0.731800,0.680667,12.0,low_season,Wednesday
1997-12-18,3,0.819167,2.835167,1.289583,10.299250,18.399250,8.888667,10.072917,10.472667,2.935583,...,5.448583,6.026500,2.498167,3.977833,0.824750,0.826833,0.824500,12.0,low_season,Thursday
1997-12-19,3,0.748833,0.996000,1.349750,4.972250,15.080083,5.327750,5.113417,0.833917,0.841833,...,1.257083,2.615583,2.369167,1.901833,0.746000,0.741500,0.739333,12.0,low_season,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998-12-11,3,0.876500,0.935167,0.871250,5.673083,7.313417,0.855750,0.934750,0.860917,0.860000,...,9.714083,3.317250,0.869583,0.866167,0.799833,0.697000,0.876250,12.0,low_season,Friday
1998-12-12,3,0.874833,0.736917,0.740750,11.167333,9.234083,7.649167,13.865083,4.780583,3.131333,...,19.115250,8.940500,9.794583,6.815083,1.174500,1.165083,1.780167,12.0,low_season,Saturday
1998-12-13,3,1.135917,1.135250,1.123833,2.085000,2.962167,7.382750,1.147667,1.152500,1.772500,...,2.161250,7.301917,7.097000,3.526000,1.123667,3.250417,1.104500,12.0,low_season,Sunday


In [381]:
measured_DF[measured_DF['ProfileID'] == 3]

,ProfileID,date,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
86002,3,1997-12-15,4.396250,0.673833,0.801250,0.808667,0.813250,0.802917,0.811750,1.048583,...,0.807417,0.734750,0.730750,0.733833,0.761167,6.857833,8.509500,3.316750,6.862917,0.888250
86003,3,1997-12-16,0.668250,0.740333,0.741583,0.799250,0.667583,0.800750,1.310417,5.723500,...,1.881417,7.523250,7.920833,13.833750,7.974583,6.810083,0.838167,0.751833,0.749083,0.750667
86004,3,1997-12-17,0.812167,0.751750,1.382000,6.623917,7.606583,9.273083,0.680833,0.818750,...,15.785666,10.799750,12.174083,2.800333,2.352500,0.992833,0.813667,0.747500,0.731800,0.680667
86005,3,1997-12-18,0.819167,2.835167,1.289583,10.299250,18.399250,8.888667,10.072917,10.472667,...,1.642833,2.727917,5.839833,5.448583,6.026500,2.498167,3.977833,0.824750,0.826833,0.824500
86006,3,1997-12-19,0.748833,0.996000,1.349750,4.972250,15.080083,5.327750,5.113417,0.833917,...,4.022417,3.201750,1.503833,1.257083,2.615583,2.369167,1.901833,0.746000,0.741500,0.739333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179673,3,1998-11-17,1.548583,1.501250,1.489250,1.489333,1.490083,4.631833,2.225833,6.796500,...,1.714000,1.644833,1.472750,6.207000,7.090250,2.373917,5.575250,5.032167,8.888000,5.906000
179674,3,1998-11-18,5.737500,1.787167,1.486583,4.700250,1.483500,1.547833,1.693333,2.636917,...,2.757250,3.946667,7.037167,11.789750,4.221500,3.375167,4.434417,4.984917,8.834000,1.211250
179675,3,1998-11-19,1.154583,1.390250,1.254000,1.325333,1.322500,1.338667,4.270000,8.597750,...,1.258167,1.320667,4.442583,6.350750,6.473583,4.125750,6.290417,5.712333,7.395583,1.237083
179676,3,1998-11-20,1.219083,1.247667,1.390917,1.148583,1.227167,1.304000,5.912250,7.909167,...,1.740167,1.810667,1.806500,4.106583,5.788333,13.644083,12.443000,19.370000,23.087333,13.245833


In [372]:
data_new

,ProfileID,0,1,2,3,4,5,6,7,8,...,17,18,19,20,21,22,23,month,season,day_names
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,12029036,2.783333,2.566667,2.100000,0.300000,1.650000,2.200000,1.866667,1.933333,4.533333,...,2.783333,2.383333,2.750000,3.000000,2.900000,2.400000,2.200000,1,low_season,Wednesday
2014-01-02,12029036,2.233333,1.766667,1.400000,1.566667,0.983333,2.000000,1.550000,2.716667,3.283333,...,2.600000,2.583333,2.716667,2.966667,2.683333,2.450000,2.300000,1,low_season,Thursday
2014-01-03,12029036,2.200000,2.200000,2.200000,1.883333,1.400000,2.183333,1.766667,2.766667,2.750000,...,2.400000,4.116667,3.783333,4.700000,4.100000,2.900000,2.516667,1,low_season,Friday
2014-01-04,12029036,2.266667,2.200000,2.200000,2.200000,1.833333,1.483333,1.650000,2.316667,3.466667,...,3.683333,2.533333,2.666667,2.966667,2.766667,2.400000,2.400000,1,low_season,Saturday
2014-01-05,12029036,2.283333,2.200000,2.200000,2.200000,1.900000,1.400000,2.450000,1.550000,2.983333,...,2.400000,3.133333,3.516667,3.183333,2.916667,2.550000,2.200000,1,low_season,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-28,109,0.462500,0.463167,0.462333,0.463000,0.463167,0.465083,1.245167,1.753333,0.873167,...,0.179333,0.914167,0.090833,4.488500,1.823250,0.429500,0.082000,12,low_season,Sunday
2014-12-29,109,0.480583,0.461250,0.461917,0.461750,0.460417,0.470000,4.358333,1.783000,3.845000,...,1.402667,0.216583,0.348083,2.760833,5.326000,0.582750,0.497667,12,low_season,Monday
2014-12-30,109,0.238583,0.131083,0.114750,0.114583,0.113417,0.113167,0.110417,2.512583,4.103167,...,1.127083,8.663917,7.802250,2.030750,0.426500,0.236917,0.171917,12,low_season,Tuesday


In [313]:
High_season = [6,7,8]
x = missing_date.month


if x in High_season:
    print('High')
else:
    print('Low')

missing_date.month

Low


1

In [306]:
data_new['season'].unique()

array(['high_season', 'low_season'], dtype=object)

In [158]:
end_date_new = start_date + relativedelta(years = 1)

In [160]:
data_new = data_new.loc[start_date:end_date_new-timedelta(days = 1)]

In [161]:
# data_new['MM-DD'] = data_new['date'].dt.strftime('%m-%d')

In [162]:
data_new.reset_index(inplace=True)

In [163]:
# data_new['MM-DD'] = pd.to_datetime(data_new['MM-DD'], errors='coerce')

In [164]:
# data_new.info()

In [165]:
data_new['YY-MM-DD'] = str(data_new['date'].dt.year[0]) + '-' + data_new['date'].dt.strftime('%m-%d')

In [166]:
data_new['YY-MM-DD'] = pd.to_datetime(data_new['YY-MM-DD'], errors = 'coerce')

In [167]:
data_new.set_index(['YY-MM-DD']).sort_index()

,date,ProfileID,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
YY-MM-DD,,,,,,,,,,,,,,,,,,,,,
1994-01-01,1995-01-01,1.0,1.800000,1.800000,1.850000,1.700000,1.850000,1.966667,1.200000,1.20,...,1.20,1.150000,1.20,1.200000,1.20,1.150000,1.20,1.200000,1.150000,1.516667
1994-01-02,1995-01-02,1.0,1.233333,1.400000,1.366667,1.400000,1.700000,1.433333,1.150000,1.20,...,1.20,1.200000,1.15,1.150000,0.90,1.200000,1.20,1.150000,1.200000,1.233333
1994-01-03,1995-01-03,1.0,1.366667,1.400000,1.650000,1.700000,1.533333,2.050000,1.150000,1.20,...,1.20,1.200000,1.15,1.200000,0.95,1.200000,1.20,1.150000,1.200000,1.266667
1994-01-04,1995-01-04,1.0,1.366667,1.400000,1.450000,1.750000,2.016667,2.533333,1.400000,1.20,...,1.15,1.200000,1.20,1.050000,1.00,1.000000,1.20,1.150000,1.366667,1.650000
1994-01-05,1995-01-05,1.0,2.250000,2.483333,2.300000,2.400000,2.366667,2.400000,2.000000,1.50,...,1.20,1.150000,1.10,1.150000,1.00,1.200000,1.15,1.266667,1.500000,1.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994-12-27,1994-12-27,1.0,1.200000,1.200000,1.150000,1.200000,1.200000,1.150000,1.200000,1.20,...,1.20,1.066667,1.15,1.200000,1.05,1.050000,1.20,1.200000,1.200000,1.150000
1994-12-28,1994-12-28,1.0,1.200000,1.200000,1.150000,1.150000,1.200000,1.466667,1.200000,1.15,...,1.15,1.150000,1.00,0.900000,0.90,1.133333,1.10,1.200000,1.150000,1.200000
1994-12-29,1994-12-29,1.0,1.200000,1.200000,1.150000,1.200000,1.200000,1.150000,1.200000,1.20,...,1.20,1.150000,1.10,0.950000,1.00,1.000000,1.20,1.200000,1.200000,1.150000
